<a href="https://colab.research.google.com/github/SarthakBMSCE/BIS_Lab_1BM22CS352/blob/main/BIS_LAB_GWO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Objective function (example: Sphere function)
def objective_function(x):
    return np.sum(x**2)

# Grey Wolf Optimizer (GWO) algorithm
class GreyWolfOptimizer:
    def __init__(self, objective_function, n_wolves, n_variables, max_iter, lb, ub):
        self.obj_func = objective_function  # Objective function
        self.n_wolves = n_wolves  # Number of wolves
        self.n_variables = n_variables  # Number of variables in the problem
        self.max_iter = max_iter  # Maximum number of iterations
        self.lb = lb  # Lower bound for the search space
        self.ub = ub  # Upper bound for the search space

        # Initialize wolves randomly in the search space
        self.wolves = np.random.uniform(self.lb, self.ub, (self.n_wolves, self.n_variables))

        # Initialize alpha, beta, delta wolves
        self.alpha = np.zeros(self.n_variables)
        self.beta = np.zeros(self.n_variables)
        self.delta = np.zeros(self.n_variables)
        self.alpha_score = float("inf")
        self.beta_score = float("inf")
        self.delta_score = float("inf")

    def update_wolves(self):
        # Evaluate the fitness of each wolf
        fitness = np.apply_along_axis(self.obj_func, 1, self.wolves)

        # Sort wolves based on fitness values
        sorted_indices = np.argsort(fitness)
        self.wolves = self.wolves[sorted_indices]
        fitness = fitness[sorted_indices]

        # Update alpha, beta, and delta wolves
        self.alpha = self.wolves[0]
        self.beta = self.wolves[1]
        self.delta = self.wolves[2]

        # Update the alpha, beta, and delta scores
        self.alpha_score = fitness[0]
        self.beta_score = fitness[1]
        self.delta_score = fitness[2]

    def optimize(self):
        for t in range(self.max_iter):
            # Update wolves based on the positions of alpha, beta, and delta wolves
            A = 2 * np.random.random((self.n_wolves, self.n_variables)) - 1  # Random values for exploration
            C = 2 * np.random.random((self.n_wolves, self.n_variables))  # Random values for exploitation

            # Update positions of wolves
            for i in range(self.n_wolves):
                D_alpha = np.abs(C[i] * self.alpha - self.wolves[i])  # Distance to alpha wolf
                D_beta = np.abs(C[i] * self.beta - self.wolves[i])  # Distance to beta wolf
                D_delta = np.abs(C[i] * self.delta - self.wolves[i])  # Distance to delta wolf

                # Update position of wolf i
                self.wolves[i] = self.alpha - A[i] * D_alpha  # Move towards alpha wolf

                # Ensure the positions stay within the bounds
                self.wolves[i] = np.clip(self.wolves[i], self.lb, self.ub)

            # Update alpha, beta, and delta wolves after each iteration
            self.update_wolves()

            # Display progress (optional)
            print(f"Iteration {t+1}/{self.max_iter}, Best Score: {self.alpha_score}")

        return self.alpha, self.alpha_score  # Return the best solution found


# Parameters
n_wolves = 30  # Number of wolves
n_variables = 5  # Number of decision variables
max_iter = 100  # Maximum number of iterations
lb = -10  # Lower bound of the search space
ub = 10  # Upper bound of the search space

# Create an instance of the GreyWolfOptimizer
gwo = GreyWolfOptimizer(objective_function, n_wolves, n_variables, max_iter, lb, ub)

# Run the optimizer
best_solution, best_score = gwo.optimize()

print("Best Solution Found:", best_solution)
print("Best Score:", best_score)


Iteration 1/100, Best Score: 5.025992655651739
Iteration 2/100, Best Score: 5.672690163345066
Iteration 3/100, Best Score: 4.09919386564303
Iteration 4/100, Best Score: 3.0664543365869434
Iteration 5/100, Best Score: 1.7116182434968712
Iteration 6/100, Best Score: 1.6448046401774825
Iteration 7/100, Best Score: 0.5347172446620828
Iteration 8/100, Best Score: 0.37386977688275225
Iteration 9/100, Best Score: 0.20059820023205482
Iteration 10/100, Best Score: 0.08732252835101455
Iteration 11/100, Best Score: 0.007001499156245157
Iteration 12/100, Best Score: 0.007556016890255042
Iteration 13/100, Best Score: 0.005086297275156763
Iteration 14/100, Best Score: 0.0021253596544082765
Iteration 15/100, Best Score: 0.0009111927589815459
Iteration 16/100, Best Score: 0.0004111387353930545
Iteration 17/100, Best Score: 0.00033091030412428876
Iteration 18/100, Best Score: 0.00024145162296968807
Iteration 19/100, Best Score: 7.191616454731122e-05
Iteration 20/100, Best Score: 4.0208570139238547e-05


In [ ]:
import numpy as np

# Objective function (example: Sphere function)
def objective_function(x):
    return np.sum(x**2)

class ParticleSwarmOptimizer:
    def __init__(self, objective_function, n_particles, n_variables, max_iter, lb, ub, w=0.5, c1=1.5, c2=1.5):
        self.obj_func = objective_function  # Objective function
        self.n_particles = n_particles  # Number of particles
        self.n_variables = n_variables  # Number of decision variables
        self.max_iter = max_iter  # Maximum number of iterations
        self.lb = lb  # Lower bound of the search space
        self.ub = ub  # Upper bound of the search space
        self.w = w  # Inertia weight
        self.c1 = c1  # Cognitive coefficient
        self.c2 = c2  # Social coefficient

        # Initialize particles (position and velocity)
        self.position = np.random.uniform(self.lb, self.ub, (self.n_particles, self.n_variables))
        self.velocity = np.random.uniform(-1, 1, (self.n_particles, self.n_variables))

        # Initialize pBest and gBest
        self.pBest = np.copy(self.position)
        self.pBest_score = np.apply_along_axis(self.obj_func, 1, self.pBest)
        self.gBest = self.pBest[np.argmin(self.pBest_score)]
        self.gBest_score = np.min(self.pBest_score)

    def optimize(self):
        for t in range(self.max_iter):
            # Update the velocity and position of each particle
            for i in range(self.n_particles):
                r1, r2 = np.random.rand(2)  # Random numbers between 0 and 1
                # Update velocity
                self.velocity[i] = (self.w * self.velocity[i] +
                                    self.c1 * r1 * (self.pBest[i] - self.position[i]) +
                                    self.c2 * r2 * (self.gBest - self.position[i]))

                # Update position
                self.position[i] = self.position[i] + self.velocity[i]

                # Apply boundary conditions (clipping positions to [lb, ub])
                self.position[i] = np.clip(self.position[i], self.lb, self.ub)

            # Evaluate fitness
            fitness = np.apply_along_axis(self.obj_func, 1, self.position)

            # Update pBest and gBest
            for i in range(self.n_particles):
                if fitness[i] < self.pBest_score[i]:
                    self.pBest_score[i] = fitness[i]
                    self.pBest[i] = self.position[i]

            # Update the global best
            min_score_idx = np.argmin(self.pBest_score)
            if self.pBest_score[min_score_idx] < self.gBest_score:
                self.gBest_score = self.pBest_score[min_score_idx]
                self.gBest = self.pBest[min_score_idx]

            # Display progress (optional)
            print(f"Iteration {t+1}/{self.max_iter}, Best Score: {self.gBest_score}")

        return self.gBest, self.gBest_score  # Return the best solution found


# Parameters
n_particles = 30  # Number of particles
n_variables = 5  # Number of decision variables
max_iter = 100  # Maximum number of iterations
lb = -10  # Lower bound of the search space
ub = 10  # Upper bound of the search space

# Create an instance of the ParticleSwarmOptimizer
pso = ParticleSwarmOptimizer(objective_function, n_particles, n_variables, max_iter, lb, ub)

# Run the optimizer
best_solution, best_score = pso.optimize()

print("Best Solution Found:", best_solution)
print("Best Score:", best_score)


Iteration 1/100, Best Score: 28.789386644470625
Iteration 2/100, Best Score: 28.090743304200522
Iteration 3/100, Best Score: 16.617866774784407
Iteration 4/100, Best Score: 0.8412233181448907
Iteration 5/100, Best Score: 0.8412233181448907
Iteration 6/100, Best Score: 0.8412233181448907
Iteration 7/100, Best Score: 0.12431831084995712
Iteration 8/100, Best Score: 0.12431831084995712
Iteration 9/100, Best Score: 0.12431831084995712
Iteration 10/100, Best Score: 0.08819891753987495
Iteration 11/100, Best Score: 0.08819891753987495
Iteration 12/100, Best Score: 0.058056107049846525
Iteration 13/100, Best Score: 0.047279073338287014
Iteration 14/100, Best Score: 0.028204912445923783
Iteration 15/100, Best Score: 0.021635285310744275
Iteration 16/100, Best Score: 0.01539503717993404
Iteration 17/100, Best Score: 0.01539503717993404
Iteration 18/100, Best Score: 0.0028219592674728227
Iteration 19/100, Best Score: 0.0026457308382116895
Iteration 20/100, Best Score: 0.0026457308382116895
Itera

In [ ]:
import numpy as np
from scipy.special import gamma  # Correct import for gamma function

# Objective function (example: Sphere function)
def objective_function(x):
    return np.sum(x**2)

class CuckooSearch:
    def __init__(self, objective_function, n_cuckoos, n_variables, max_iter, lb, ub, pa=0.25):
        self.obj_func = objective_function  # Objective function
        self.n_cuckoos = n_cuckoos  # Number of cuckoos (solutions)
        self.n_variables = n_variables  # Number of decision variables
        self.max_iter = max_iter  # Maximum number of iterations
        self.lb = lb  # Lower bound of the search space
        self.ub = ub  # Upper bound of the search space
        self.pa = pa  # Probability of abandoning a nest (fraction of bad nests)

        # Initialize cuckoos randomly in the search space
        self.cuckoos = np.random.uniform(self.lb, self.ub, (self.n_cuckoos, self.n_variables))
        self.fitness = np.apply_along_axis(self.obj_func, 1, self.cuckoos)

        # Keep track of the best cuckoo (solution)
        self.best_solution = self.cuckoos[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)

    def levy_flight(self):
        """Generate a step using Lévy flight"""
        beta = 1.5  # Lévy flight parameter
        sigma = (gamma(1 + beta) * np.sin(np.pi * beta / 2) / gamma((1 + beta) / 2)
                 * np.cos(np.pi * beta / 2) ** (beta / 2)) ** (1 / beta)
        step = np.random.normal(0, sigma, self.n_variables) * np.random.standard_normal(self.n_variables)
        return step

    def optimize(self):
        for t in range(self.max_iter):
            # Generate new cuckoos (new solutions) using Lévy flight
            new_cuckoos = np.copy(self.cuckoos)
            for i in range(self.n_cuckoos):
                step = self.levy_flight()
                new_solution = self.cuckoos[i] + step
                new_solution = np.clip(new_solution, self.lb, self.ub)  # Ensure within bounds

                # Evaluate fitness of the new solution
                new_fitness = self.obj_func(new_solution)

                # If the new solution is better, replace the old one
                if new_fitness < self.fitness[i]:
                    self.cuckoos[i] = new_solution
                    self.fitness[i] = new_fitness

            # Find the best cuckoo in the population
            best_cuckoo_idx = np.argmin(self.fitness)
            if self.fitness[best_cuckoo_idx] < self.best_fitness:
                self.best_solution = self.cuckoos[best_cuckoo_idx]
                self.best_fitness = self.fitness[best_cuckoo_idx]

            # Abandon a fraction of worst nests and generate new ones randomly
            # Abandon nests based on the probability 'pa'
            num_abandoned = int(self.pa * self.n_cuckoos)
            worst_cuckoos = np.argsort(self.fitness)[-num_abandoned:]
            for i in worst_cuckoos:
                self.cuckoos[i] = np.random.uniform(self.lb, self.ub, self.n_variables)
                self.fitness[i] = self.obj_func(self.cuckoos[i])

            # Display progress (optional)
            print(f"Iteration {t+1}/{self.max_iter}, Best Score: {self.best_fitness}")

        return self.best_solution, self.best_fitness  # Return the best solution found

# Parameters
n_cuckoos = 30  # Number of cuckoos (solutions)
n_variables = 5  # Number of decision variables
max_iter = 100  # Maximum number of iterations
lb = -10  # Lower bound of the search space
ub = 10  # Upper bound of the search space

# Create an instance of the CuckooSearch class
csa = CuckooSearch(objective_function, n_cuckoos, n_variables, max_iter, lb, ub)

# Run the optimizer
best_solution, best_score = csa.optimize()

print("Best Solution Found:", best_solution)
print("Best Score:", best_score)


<ipython-input-10-16abbbbb833e>:30: RuntimeWarning: invalid value encountered in scalar power
  * np.cos(np.pi * beta / 2) ** (beta / 2)) ** (1 / beta)


Iteration 1/100, Best Score: 39.57197556707513
Iteration 2/100, Best Score: 39.57197556707513
Iteration 3/100, Best Score: 39.57197556707513
Iteration 4/100, Best Score: 39.57197556707513
Iteration 5/100, Best Score: 39.57197556707513
Iteration 6/100, Best Score: 39.57197556707513
Iteration 7/100, Best Score: 39.57197556707513
Iteration 8/100, Best Score: 39.57197556707513
Iteration 9/100, Best Score: 31.51352811791149
Iteration 10/100, Best Score: 31.51352811791149
Iteration 11/100, Best Score: 31.51352811791149
Iteration 12/100, Best Score: 31.51352811791149
Iteration 13/100, Best Score: 31.51352811791149
Iteration 14/100, Best Score: 31.51352811791149
Iteration 15/100, Best Score: 31.51352811791149
Iteration 16/100, Best Score: 31.51352811791149
Iteration 17/100, Best Score: 31.51352811791149
Iteration 18/100, Best Score: 31.51352811791149
Iteration 19/100, Best Score: 31.51352811791149
Iteration 20/100, Best Score: 31.51352811791149
Iteration 21/100, Best Score: 31.51352811791149
I